In [17]:
# Install required libraries
!pip install folium polyline requests
import folium
import polyline
import requests

# Updated journey data with corrected locations
journey_data = [
    {"city": "Bamock", "country": "Cameroon", "coords": (5.9333, 10.1667)},
    {"city": "Bambui", "country": "Cameroon", "coords": (5.9075, 10.1633)},
    {"city": "Bambili", "country": "Cameroon", "coords": (5.9167, 10.1833)},
    {"city": "Bali", "country": "Cameroon", "coords": (5.8875, 10.0119)},
    {"city": "Batibo", "country": "Cameroon", "coords": (5.84080649952068, 9.877052215339347)},
    {"city": "Widikum", "country": "Cameroon", "coords": (5.867442112679804, 9.766460462538019)},
    {"city": "Mamfe", "country": "Cameroon", "coords": (5.7667, 9.2833)},
    {"city": "Ekok", "country": "Cameroon", "coords": (5.8108355874150295, 8.856182076570322)},
    {"city": "Ikom", "country": "Nigeria", "coords": (5.9667, 8.7167)},
    {"city": "Makurdi", "country": "Nigeria", "coords": (7.7333, 8.5333)},
    {"city": "Kano", "country": "Nigeria", "coords": (12.0000, 8.5167)},
    {"city": "Zinder", "country": "Niger", "coords": (13.8053, 8.9883)},
    {"city": "Agadez", "country": "Niger", "coords": (16.9959, 7.9858)},
    {"city": "Arlit", "country": "Niger", "coords": (18.74786952510697, 7.384507261726617)},
    {"city": "Assamakka", "country": "Algeria", "coords": (19.335786250032864, 5.772113855796544)},
    {"city": "In Guezzam", "country": "Algeria", "coords": (19.5667, 5.7667)},
    {"city": "Arak", "country": "Algeria", "coords": (25.2833, 3.7500)},
    {"city": "Aoulef", "country": "Algeria", "coords": (26.9667, 1.0833)},
    {"city": "Béchar", "country": "Algeria", "coords": (31.6167, -2.2167)},
    {"city": "Beni Ounif", "country": "Algeria", "coords": (32.0500, -1.2500)},
    {"city": "Figuig", "country": "Morocco", "coords": (32.1167, -1.2333)},
    {"city": "Ahfir", "country": "Morocco", "coords": (34.9500, -2.1000)},
    {"city": "San Rogue", "country": "Spain", "coords": (36.7350, -3.9200)}
]

# Create map with standard Google Maps style
m = folium.Map(location=[18, 5], zoom_start=3, tiles="OpenStreetMap")

# Function to get route coordinates using OSRM
def get_route_coords(start, end, waypoints=None):
    if waypoints:
        coordinates = f"{start[1]},{start[0]};"
        for wp in waypoints:
            coordinates += f"{wp[1]},{wp[0]};"
        coordinates += f"{end[1]},{end[0]}"
    else:
        coordinates = f"{start[1]},{start[0]};{end[1]},{end[0]}"

    url = f"http://router.project-osrm.org/route/v1/driving/{coordinates}?overview=full"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return polyline.decode(data['routes'][0]['geometry'])
    else:
        print(f"Error fetching route from {start} to {end}: {response.status_code}")
        return []

# Add highway-based route with manual waypoints
route_segments = [
    {"start": 0, "end": 1},  # Bamock to Bambui
    {"start": 1, "end": 2},  # Bambui to Bambili
    {"start": 2, "end": 3},  # Bambili to Bali
    {"start": 3, "end": 4},  # Bali to Batibo
    {"start": 4, "end": 5},  # Batibo to Widikum
    {"start": 5, "end": 6},  # Widikum to Mamfe
    {"start": 6, "end": 7},  # Mamfe to Ekok
    {"start": 7, "end": 8},  # Ekok to Ikom
    {"start": 8, "end": 9},  # Ikom to Makurdi
    {"start": 9, "end": 10},  # Makurdi to Kano
    {"start": 10, "end": 11},  # Kano to Zinder
    {"start": 11, "end": 12},  # Zinder to Agadez
    {"start": 12, "end": 13},  # Agadez to Arlit
    {"start": 13, "end": 14},  # Arlit to Assamakka (via waypoint)
    {"start": 14, "end": 15},  # Assamakka to In Guezzam
    {"start": 15, "end": 16},  # In Guezzam to Arak
    {"start": 16, "end": 17},  # Arak to Aoulef
    {"start": 17, "end": 18},  # Aoulef to Béchar
    {"start": 18, "end": 19},  # Béchar to Beni Ounif (via waypoint)
    {"start": 19, "end": 20},  # Beni Ounif to Figuig
    {"start": 20, "end": 21},  # Figuig to Ahfir
    {"start": 21, "end": 22},  # Ahfir to San Rogue
]

for segment in route_segments:
    start = journey_data[segment["start"]]["coords"]
    end = journey_data[segment["end"]]["coords"]
    waypoints = segment.get("waypoints", None)
    route_coords = get_route_coords(start, end, waypoints)
    if route_coords:
        folium.PolyLine(
            route_coords,
            color="#1f78b4",
            weight=3,
            opacity=0.8,
            dash_array="5,5"
        ).add_to(m)

# Create icon sequence
icon_sequence = {
    "start": {"color": "green", "icon": "home"},
    "transit": {"color": "orange", "icon": "circle"},
    "end": {"color": "red", "icon": "flag"}
}

# Add detailed markers
for idx, point in enumerate(journey_data):
    # Determine marker type
    if idx == 0:
        icon_type = "start"
    elif idx == len(journey_data)-1:
        icon_type = "end"
    else:
        icon_type = "transit"

    # Custom popup content
    popup_html = f"""
    <div style="width: 250px; font-family: Arial">
        <h4 style="color: #2c3e50; margin: 5px 0; border-bottom: 2px solid #1f78b4">
            {point['city']}, {point['country']}
        </h4>
        <p style="font-size: 14px; color: #34495e">
            <b>Chapter {idx+1}</b><br>
            Insert event
        </p>
        <div style="background: #ecf0f1; padding: 5px; margin-top: 10px">
            <small>Coordinates: {point['coords'][0]:.4f}°N, {point['coords'][1]:.4f}°E</small>
        </div>
    </div>
    """

    folium.Marker(
        location=point["coords"],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(
            color=icon_sequence[icon_type]["color"],
            icon=icon_sequence[icon_type]["icon"],
            prefix="fa"
        ),
        tooltip=f"{point['city']} ({point['country']})"
    ).add_to(m)

# Add map title
title_html = '''
<div style="position: fixed;
     top: 10px; left: 50px; width: 450px;
     background: rgba(255,255,255,0.8); padding: 10px;
     border-radius: 5px; z-index: 1000">
    <h2 style="color: #2c3e50; margin:0; font-family: Arial">
        <i class="fa fa-road" style="color: #1f78b4"></i>
        DRIVE NORTH: Migration Route Map
    </h2>
    <p style="color: #7f8c8d; margin:5px 0 0 0; font-size:12px">
        From Cameroon to Spain | 4,200 km over 23 locations
    </p>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Save and display
m.save('drive_north_highways_fixed.html')
from google.colab import files
files.download('drive_north_highways_fixed.html')

# For Colab display
from IPython.display import HTML
HTML('<iframe src=drive_north_highways_fixed.html width=900 height=600></iframe>')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>